# Flujo de ventas de juegos


## Se van a escoger 700 usuarios de los 800 existentes para que compren entre 1-5 juegos en diferentes fechas

In [2]:
import random
import mysql.connector
import json
import string
from pymongo import MongoClient
import datetime as dt


def random_text(valoracion):
    switcher = {        
        1: "No lo recomiendo",
        2: "Los peores euros invertidos de mi vida",
        3: "Decepcionante",
        4: "Me esperaba algo mejor",
        5: "No esta mal",
        6: "Es entretenido",
        7: "Me ha gustado",
        8: "Simplemente perfecto",
        9: "De los mejores juegos que he jugado",
        10: "Para mi, el juego del año"
    }
    return switcher.get(valoracion, valoracion)    
   
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient['stum_for_you']
mycol = mydb["usuarios"]

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    database='stum_for_you',
    passwd=""
)

mycursor = mydb.cursor()



for i in range(1,701):
    #vamos a ver cuantos juegos quiere comprar el usuario
    numjue = random.randrange(2,6)
    for a in range(1,numjue):
        #Procedemos a comprobar que claves tenemos no vendidas
        mycursor.execute("SELECT c.id_clave FROM claves_juegos c WHERE vendido = 0")
        claves_libres = mycursor.fetchall()
        id = random.randrange(0,len(claves_libres))
        #sacamos una clave de entre las no vendidas
        id_clave=claves_libres[id]
        id_clave=id_clave[0]
        mycursor.execute("SELECT id_juego, fecha_anexion FROM claves_juegos WHERE id_clave = %s", (id_clave,))
        datos = mycursor.fetchone()
        #obtenemos el id del juego perteneciente a la clave
        id_juego =datos[0]
        #ponemos que el usuario compro dicha clave 4 semanas de su compra por ejemplo
        fecha = (datos[1])+dt.timedelta(weeks=+4)
        mycursor.execute("SELECT precio FROM juegos WHERE id_juego = %s", (id_juego,))
        preciojuego = mycursor.fetchone()
        mycursor.execute("SELECT porcentaje_a_aplicar,fecha_creacion,fecha_vencimiento FROM descuentos WHERE id_juego = %s", (id_juego,))
        descuentos = mycursor.fetchall()
        des=0
        #COMPROBAMOS AHORA SI EXISTE DESCUENTO PARA DICHO Juego
        for descuento in descuentos:
            fechaini=descuento[1]
            fechafin=descuento[2]
            des=descuento[0]
            if(fecha>fechaini and fecha<fechafin):
                break
        print("precio ",preciojuego[0])
        print("descuento ",des)
        
        preciojuego = int(preciojuego[0]-(preciojuego[0] * (des/100)))
        print("precio final ",preciojuego)
        print("fecha  ",fecha)
        usuario=i
        c={}
        c["id_usuario"] = usuario
        usuario_db = mycol.find_one(c)
        biblioteca = {}
        update = {}
        biblioteca = usuario_db['biblioteca']
        if biblioteca == 0:
            biblioteca = []
        juego={}
        juego['id_juego'] = id_juego
        juego['valoraciones'] = random.randrange(1,11)
        juego['comentarios'] = random_text(juego['valoraciones'])
        biblioteca.append(juego)
        update["$set"] = {}
        update["$set"]['biblioteca'] = biblioteca
        a=mycol.update_one({ "id_usuario": usuario }, update)
        #print (a.modified_count)
       # print((usuario, preciojuego))
        if(a.modified_count==1):
            #vamos a actualizar la clave, modificando el estado a vendido
            mycursor.execute("UPDATE claves_juegos SET vendido = 1 WHERE id_clave = %s", (id_clave,))
            #ahora añadimos la transacion
            sql = "INSERT INTO transacciones_venta (precio_total, fecha_venta) VALUES (%s, %s)"
            val = (preciojuego, fecha)
            mycursor.execute(sql,val)

            mycursor.execute("SELECT MAX(id_transaccion) FROM transacciones_venta")
            id_transac = mycursor.fetchone()[0]
            sql = "INSERT INTO venta_juegos (id_usuario, id_transaccion, id_claves_juego, precio) VALUES (%s, %s, %s, %s)"
           # print((usuario, id_transac, id_clave, preciojuego))
            val = (usuario, id_transac, id_clave, preciojuego)
            mycursor.execute(sql,val)
            mydb.commit() 

precio  15
descuento  0
precio final  15
fecha   2019-06-03
precio  5
descuento  84
precio final  0
fecha   2020-01-25
precio  3
descuento  0
precio final  3
fecha   2019-07-07
precio  21
descuento  0
precio final  21
fecha   2017-11-08
precio  15
descuento  0
precio final  15
fecha   2019-12-25
precio  5
descuento  0
precio final  5
fecha   2017-10-22
precio  7
descuento  0
precio final  7
fecha   2017-05-17
precio  0
descuento  0
precio final  0
fecha   2019-08-10
precio  0
descuento  0
precio final  0
fecha   2020-01-11
precio  13
descuento  0
precio final  13
fecha   2017-08-14
precio  25
descuento  0
precio final  25
fecha   2017-03-06
precio  10
descuento  0
precio final  10
fecha   2019-12-22
precio  15
descuento  0
precio final  15
fecha   2019-06-14
precio  25
descuento  0
precio final  25
fecha   2019-09-25
precio  0
descuento  0
precio final  0
fecha   2019-07-11
precio  15
descuento  0
precio final  15
fecha   2019-11-11
precio  10
descuento  0
precio final  10
fecha   2018

precio  2
descuento  0
precio final  2
fecha   2019-08-23
precio  13
descuento  0
precio final  13
fecha   2018-09-11
precio  0
descuento  0
precio final  0
fecha   2018-01-23
precio  10
descuento  0
precio final  10
fecha   2019-12-22
precio  10
descuento  0
precio final  10
fecha   2018-03-03
precio  5
descuento  0
precio final  5
fecha   2019-12-16
precio  0
descuento  0
precio final  0
fecha   2019-05-04
precio  3
descuento  0
precio final  3
fecha   2019-07-28
precio  3
descuento  0
precio final  3
fecha   2017-10-08
precio  10
descuento  0
precio final  10
fecha   2020-01-13
precio  10
descuento  0
precio final  10
fecha   2019-11-17
precio  10
descuento  0
precio final  10
fecha   2018-10-16
precio  5
descuento  0
precio final  5
fecha   2019-11-28
precio  25
descuento  0
precio final  25
fecha   2017-12-14
precio  20
descuento  0
precio final  20
fecha   2018-04-01
precio  10
descuento  0
precio final  10
fecha   2020-01-09
precio  5
descuento  0
precio final  5
fecha   2019-05

precio  10
descuento  0
precio final  10
fecha   2018-03-03
precio  5
descuento  85
precio final  0
fecha   2019-04-02
precio  15
descuento  0
precio final  15
fecha   2018-04-01
precio  15
descuento  0
precio final  15
fecha   2018-01-16
precio  10
descuento  0
precio final  10
fecha   2018-10-16
precio  20
descuento  0
precio final  20
fecha   2018-04-01
precio  3
descuento  0
precio final  3
fecha   2020-01-28
precio  10
descuento  0
precio final  10
fecha   2018-06-01
precio  15
descuento  0
precio final  15
fecha   2017-03-10
precio  4
descuento  0
precio final  4
fecha   2018-04-12
precio  3
descuento  0
precio final  3
fecha   2019-03-28
precio  10
descuento  0
precio final  10
fecha   2018-06-03
precio  10
descuento  0
precio final  10
fecha   2019-12-22
precio  0
descuento  0
precio final  0
fecha   2019-12-12
precio  5
descuento  0
precio final  5
fecha   2017-09-04
precio  4
descuento  0
precio final  4
fecha   2019-03-18
precio  2
descuento  0
precio final  2
fecha   2019-0

precio  18
descuento  0
precio final  18
fecha   2019-08-23
precio  5
descuento  0
precio final  5
fecha   2018-02-02
precio  4
descuento  0
precio final  4
fecha   2018-10-05
precio  30
descuento  39
precio final  18
fecha   2019-04-29
precio  15
descuento  0
precio final  15
fecha   2018-04-26
precio  2
descuento  0
precio final  2
fecha   2019-12-26
precio  10
descuento  0
precio final  10
fecha   2019-09-27
precio  10
descuento  0
precio final  10
fecha   2019-07-08
precio  15
descuento  0
precio final  15
fecha   2019-04-19
precio  7
descuento  0
precio final  7
fecha   2017-09-29
precio  21
descuento  0
precio final  21
fecha   2017-11-08
precio  7
descuento  0
precio final  7
fecha   2017-04-11
precio  10
descuento  0
precio final  10
fecha   2018-05-29
precio  4
descuento  0
precio final  4
fecha   2019-03-18
precio  40
descuento  0
precio final  40
fecha   2020-01-27
precio  4
descuento  0
precio final  4
fecha   2020-01-01
precio  10
descuento  0
precio final  10
fecha   2019

precio  1
descuento  18
precio final  0
fecha   2019-04-01
precio  10
descuento  0
precio final  10
fecha   2018-10-16
precio  0
descuento  0
precio final  0
fecha   2019-05-16
precio  15
descuento  0
precio final  15
fecha   2019-06-14
precio  5
descuento  0
precio final  5
fecha   2019-12-27
precio  5
descuento  0
precio final  5
fecha   2017-09-04
precio  5
descuento  0
precio final  5
fecha   2019-04-10
precio  50
descuento  0
precio final  50
fecha   2019-08-20
precio  20
descuento  0
precio final  20
fecha   2018-04-01
precio  4
descuento  0
precio final  4
fecha   2018-04-12
precio  20
descuento  0
precio final  20
fecha   2019-12-29
precio  13
descuento  0
precio final  13
fecha   2017-08-14
precio  0
descuento  0
precio final  0
fecha   2019-02-15
precio  10
descuento  0
precio final  10
fecha   2019-06-04
precio  0
descuento  0
precio final  0
fecha   2019-10-02
precio  15
descuento  0
precio final  15
fecha   2019-03-20
precio  20
descuento  0
precio final  20
fecha   2018-0

...

precio  25
descuento  0
precio final  25
fecha   2017-12-14
precio  20
descuento  0
precio final  20
fecha   2018-04-01
precio  13
descuento  0
precio final  13
fecha   2019-10-13
precio  10
descuento  0
precio final  10
fecha   2019-07-10
precio  10
descuento  0
precio final  10
fecha   2017-02-28
precio  3
descuento  0
precio final  3
fecha   2019-08-01
precio  0
descuento  0
precio final  0
fecha   2019-05-17
precio  10
descuento  0
precio final  10
fecha   2018-12-01
precio  3
descuento  0
precio final  3
fecha   2019-08-13
precio  10
descuento  0
precio final  10
fecha   2019-12-22
precio  2
descuento  0
precio final  2
fecha   2019-07-31
precio  10
descuento  0
precio final  10
fecha   2017-04-07
precio  2
descuento  0
precio final  2
fecha   2017-08-23
precio  7
descuento  0
precio final  7
fecha   2019-04-26
precio  25
descuento  0
precio final  25
fecha   2017-03-06
precio  20
descuento  0
precio final  20
fecha   2019-06-26
precio  10
descuento  0
precio final  10
fecha   201

precio  5
descuento  0
precio final  5
fecha   2019-06-28
precio  5
descuento  0
precio final  5
fecha   2017-09-04
precio  10
descuento  0
precio final  10
fecha   2019-12-22
precio  20
descuento  0
precio final  20
fecha   2017-08-14
precio  13
descuento  0
precio final  13
fecha   2019-02-26
precio  3
descuento  0
precio final  3
fecha   2019-11-12
precio  13
descuento  0
precio final  13
fecha   2017-08-14
precio  5
descuento  0
precio final  5
fecha   2018-02-02
precio  0
descuento  0
precio final  0
fecha   2019-11-30
precio  10
descuento  0
precio final  10
fecha   2017-05-30
precio  15
descuento  0
precio final  15
fecha   2018-01-16
precio  2
descuento  0
precio final  2
fecha   2019-08-23
precio  15
descuento  0
precio final  15
fecha   2019-04-19
precio  7
descuento  0
precio final  7
fecha   2019-10-12
precio  8
descuento  0
precio final  8
fecha   2017-07-09
precio  0
descuento  0
precio final  0
fecha   2020-01-17
precio  0
descuento  0
precio final  0
fecha   2019-08-10


precio  0
descuento  0
precio final  0
fecha   2020-01-11
precio  15
descuento  0
precio final  15
fecha   2019-03-31
precio  0
descuento  0
precio final  0
fecha   2018-03-03
precio  13
descuento  0
precio final  13
fecha   2019-06-23
precio  7
descuento  0
precio final  7
fecha   2018-12-15
precio  3
descuento  0
precio final  3
fecha   2019-12-25
precio  5
descuento  0
precio final  5
fecha   2019-03-31
precio  9
descuento  0
precio final  9
fecha   2018-07-03
precio  50
descuento  0
precio final  50
fecha   2019-08-20
precio  10
descuento  0
precio final  10
fecha   2019-12-22
precio  5
descuento  0
precio final  5
fecha   2018-01-02
precio  2
descuento  0
precio final  2
fecha   2019-08-23
precio  7
descuento  0
precio final  7
fecha   2019-08-31
precio  18
descuento  0
precio final  18
fecha   2019-10-07
precio  7
descuento  0
precio final  7
fecha   2017-09-29
precio  15
descuento  0
precio final  15
fecha   2019-11-03
precio  10
descuento  0
precio final  10
fecha   2017-04-07


precio  1
descuento  0
precio final  1
fecha   2019-09-12
precio  30
descuento  22
precio final  23
fecha   2019-07-13
precio  10
descuento  0
precio final  10
fecha   2019-12-22
precio  10
descuento  0
precio final  10
fecha   2019-12-08
precio  5
descuento  0
precio final  5
fecha   2019-02-17
precio  5
descuento  0
precio final  5
fecha   2018-01-02
precio  10
descuento  0
precio final  10
fecha   2019-05-18
precio  10
descuento  0
precio final  10
fecha   2018-03-03
precio  20
descuento  0
precio final  20
fecha   2017-08-14
precio  18
descuento  0
precio final  18
fecha   2019-10-07
precio  13
descuento  0
precio final  13
fecha   2017-08-14
precio  2
descuento  0
precio final  2
fecha   2019-08-14
precio  0
descuento  0
precio final  0
fecha   2019-10-19
precio  0
descuento  0
precio final  0
fecha   2020-01-11
precio  10
descuento  0
precio final  10
fecha   2019-07-10
precio  5
descuento  85
precio final  0
fecha   2019-04-02
precio  16
descuento  0
precio final  16
fecha   201

precio  0
descuento  0
precio final  0
fecha   2019-07-11
precio  0
descuento  0
precio final  0
fecha   2019-02-17
precio  3
descuento  0
precio final  3
fecha   2019-03-28
precio  0
descuento  0
precio final  0
fecha   2020-01-19
precio  5
descuento  92
precio final  0
fecha   2019-02-07
precio  0
descuento  0
precio final  0
fecha   2019-09-07
precio  0
descuento  0
precio final  0
fecha   2019-03-07
precio  10
descuento  0
precio final  10
fecha   2019-08-30
precio  0
descuento  0
precio final  0
fecha   2019-06-06
precio  10
descuento  0
precio final  10
fecha   2018-03-03
precio  20
descuento  0
precio final  20
fecha   2019-04-05
precio  4
descuento  0
precio final  4
fecha   2018-04-12
precio  0
descuento  0
precio final  0
fecha   2020-01-11
precio  7
descuento  0
precio final  7
fecha   2019-04-02
precio  6
descuento  0
precio final  6
fecha   2019-10-27
precio  8
descuento  0
precio final  8
fecha   2019-03-04
precio  5
descuento  0
precio final  5
fecha   2019-07-27
precio 

# Flujo de ventas de DLC sobre los usuarios anteriores

## Para cada usuario que tiene juegos se va generar un random de 0 o 1 para cada juego, si sale 1 procederá a comprar un dlc para ese juego si es que tiene

In [3]:
import random
import mysql.connector
import json
import string
from pymongo import MongoClient
import datetime as dt

def random_text(valoracion):
    switcher = {        
        1: "No lo recomiendo",
        2: "Los peores euros invertidos de mi vida",
        3: "Decepcionante",
        4: "Me esperaba algo mejor",
        5: "No esta mal",
        6: "Es entretenido",
        7: "Me ha gustado",
        8: "Es lo que le faltaba al juego",
        9: "De los mejores DLC que he comprado",
        10: "Si tienes el juego, tienes que comprarlo"
    }
    return switcher.get(valoracion, valoracion) 
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient['stum_for_you']
mycol = mydb["usuarios"]

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    database='stum_for_you',
    passwd=""
)

mycursor = mydb.cursor(buffered=True)



for i in range(1,701):
    #procedemos a sacar los juegos que tiene el usuario
    usuario=i
    c["id_usuario"] = usuario
    usuario_db = mycol.find_one(c)
    #recorremos los juegos del usuario
    for jue in usuario_db["biblioteca"]:
        id_juego=jue["id_juego"]
        #compra=random.randrange(0,2)
        compra=1 #forzamos a que todos quieran comprar dlc si es que su juego tiene
        if(compra):
            #si el usuario quiere comprar dlc, selecionamos el id del dlc si es que tiene
            mycursor.execute("SELECT id_dlc,precio FROM dlcs WHERE id_juego = %s", (id_juego,))
            datos = mycursor.fetchone()
            #si existe dlc para ese juego procedemos a comprarlo
            if(datos):
                id_dlc=datos[0]
                precio=datos[1]
                #comprobamos que existan claves disponibles para ese DLC
                mycursor.execute("SELECT id_clave,fecha_anexion FROM claves_dlc  WHERE id_dlc = %s; ", (id_dlc,))
                dat=mycursor.fetchone()
                if(dat):
                    id_clave=dat[0]
                    fecha = (dat[1])+dt.timedelta(weeks=+4)
                    #ahora procedemos a comprobar si existe algun descuento para dicho dlc
                    mycursor.execute("SELECT porcentaje_a_aplicar,fecha_creacion,fecha_vencimiento FROM descuentos WHERE id_dlc = %s", (id_dlc,))
                    descuentos = mycursor.fetchall()
                    des=0
                    #COMPROBAMOS AHORA SI EXISTE DESCUENTO PARA DICHO DLC
                    for descuento in descuentos:
                        fechaini=descuento[1]
                        fechafin=descuento[2]
                        descuento=descuento[0]
                        if(fecha>fechaini and fecha<fechafin):
                            break
                    print("precio ",precio)
                    print("descuento ",des)
                    preciodlc = int(precio-(precio * (des/100)))
                    print("precio final ",preciodlc)
                    print("fecha  ",fecha)
                    #creamos el formato para añadir el dlc dentro del juego correspondiente del usuario
                    dlc={}
                    dlc['id_dlc'] = id_dlc
                    dlc['valoraciones'] = random.randrange(1,11)
                    dlc['comentarios'] = random_text(dlc['valoraciones'])
                    a=mycol.update_one({ "id_usuario": usuario,"biblioteca.id_juego":id_juego },{
                        '$push': {
                            'biblioteca.$.dlcs' : dlc
                        }
                    })
                    print (a.modified_count)
                    #si se ha añadido correctamente procedemos a insertar en transacciones y en ventas el movimiento
                    #también actualizamos la tabla de claves dlc, ya que la clave del dlc vendido pasa a estado vendido
                    if(a.modified_count==1):
                        sql = "INSERT INTO transacciones_venta (precio_total, fecha_venta) VALUES (%s, %s)"
                        val = (preciodlc, fecha)
                        mycursor.execute(sql,val)
    
                        #vamos a actualizar la clave, modificando el estado a vendido
                        mycursor.execute("UPDATE claves_dlc SET vendido = 1 WHERE id_clave = %s", (id_clave,))
                
                        mycursor.execute("SELECT MAX(id_transaccion) FROM transacciones_venta")
                        id_transac = mycursor.fetchone()[0]
                        sql = "INSERT INTO venta_dlcs (id_usuario, id_tansaccion, id_claves_dlc,precio  ) VALUES (%s, %s, %s, %s)"
                        print((usuario, id_transac, id_clave, preciodlc))
                        val = (usuario, id_transac, id_clave, preciodlc)
                        mycursor.execute(sql,val)
                        mydb.commit()

precio  15
descuento  0
precio final  15
fecha   2019-05-12
1
(4, 1765, 387, 15)
precio  11
descuento  0
precio final  11
fecha   2019-04-03
1
(22, 1766, 557, 11)
precio  23
descuento  0
precio final  23
fecha   2019-04-06
1
(28, 1767, 144, 23)
precio  20
descuento  0
precio final  20
fecha   2019-04-26
1
(43, 1768, 879, 20)
precio  23
descuento  0
precio final  23
fecha   2019-04-06
1
(51, 1769, 144, 23)
precio  15
descuento  0
precio final  15
fecha   2019-09-04
1
(65, 1770, 296, 15)
precio  23
descuento  0
precio final  23
fecha   2019-04-06
1
(110, 1771, 144, 23)
precio  11
descuento  0
precio final  11
fecha   2019-04-03
1
(151, 1772, 557, 11)
precio  30
descuento  0
precio final  30
fecha   2019-03-28
1
(173, 1773, 78, 30)
precio  23
descuento  0
precio final  23
fecha   2019-04-06
1
(187, 1774, 144, 23)
precio  11
descuento  0
precio final  11
fecha   2019-07-28
1
(190, 1775, 480, 11)
precio  23
descuento  0
precio final  23
fecha   2019-04-06
1
(198, 1776, 144, 23)
precio  15
d

In [ ]:
import pandas as pd
from pymongo import MongoClient
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient['stum_for_you']
mycol = mydb["usuarios"]

df =  pd.DataFrame(list(mycol.find()))
df.to_csv (r'usuarios.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path